# Data Analysis with sqlite-worker

This notebook demonstrates how to use sqlite-worker for data analysis tasks.

## 1. Setup and Installation

In [ ]:
# Install required packages (uncomment if needed)
# !pip install sqlite-worker pandas matplotlib seaborn

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlite_worker import SqliteWorker
from datetime import datetime, timedelta
import numpy as np

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## 2. Initialize Database Connection

In [ ]:
# Initialize sqlite-worker
worker = SqliteWorker(
    'analysis.db',
    execute_init=[
        "PRAGMA journal_mode=WAL;",
        "PRAGMA synchronous=NORMAL;",
    ]
)

print("✅ Database connected")

## 3. Create Sample Dataset

In [ ]:
# Create sample sales data table
worker.execute("""
    CREATE TABLE IF NOT EXISTS sales (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        date TEXT NOT NULL,
        product TEXT NOT NULL,
        category TEXT NOT NULL,
        quantity INTEGER NOT NULL,
        price REAL NOT NULL,
        revenue REAL NOT NULL
    )
""")

# Generate sample data
categories = ['Electronics', 'Clothing', 'Food', 'Books']
products = ['Product A', 'Product B', 'Product C', 'Product D']

# Check if data exists
token = worker.execute("SELECT COUNT(*) FROM sales")
count = worker.fetch_results(token)[0][0]

if count == 0:
    # Insert sample data
    with worker.transaction():
        for i in range(1000):
            date = (datetime.now() - timedelta(days=np.random.randint(0, 365))).strftime('%Y-%m-%d')
            product = np.random.choice(products)
            category = np.random.choice(categories)
            quantity = np.random.randint(1, 20)
            price = round(np.random.uniform(10, 500), 2)
            revenue = round(quantity * price, 2)
            
            worker.insert('sales', {
                'date': date,
                'product': product,
                'category': category,
                'quantity': quantity,
                'price': price,
                'revenue': revenue
            })
    print("✅ Sample data created")
else:
    print(f"✅ Using existing data ({count} records)")

## 4. Load Data into Pandas

In [ ]:
# Query all data
token = worker.execute("SELECT * FROM sales")
data = worker.fetch_results(token)

# Convert to DataFrame
df = pd.DataFrame(data, columns=['id', 'date', 'product', 'category', 'quantity', 'price', 'revenue'])
df['date'] = pd.to_datetime(df['date'])

# Display first few rows
df.head()

## 5. Basic Statistics

In [ ]:
# Summary statistics
print("Dataset Overview:")
print(f"Total Records: {len(df)}")
print(f"Date Range: {df['date'].min()} to {df['date'].max()}")
print(f"Total Revenue: ${df['revenue'].sum():,.2f}")
print(f"Average Revenue per Sale: ${df['revenue'].mean():.2f}")

# Statistical summary
df[['quantity', 'price', 'revenue']].describe()

## 6. Sales by Category

In [ ]:
# Aggregate by category
category_sales = df.groupby('category')['revenue'].sum().sort_values(ascending=False)

# Plot
plt.figure(figsize=(10, 6))
category_sales.plot(kind='bar', color='skyblue')
plt.title('Total Revenue by Category', fontsize=14, fontweight='bold')
plt.xlabel('Category')
plt.ylabel('Revenue ($)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print("\nRevenue by Category:")
print(category_sales)

## 7. Time Series Analysis

In [ ]:
# Daily revenue
daily_revenue = df.groupby(df['date'].dt.date)['revenue'].sum()

# Plot
plt.figure(figsize=(14, 6))
plt.plot(daily_revenue.index, daily_revenue.values, linewidth=2)
plt.title('Daily Revenue Trend', fontsize=14, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Revenue ($)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 8. Product Performance

In [ ]:
# Product analysis
product_stats = df.groupby('product').agg({
    'revenue': 'sum',
    'quantity': 'sum',
    'id': 'count'
}).rename(columns={'id': 'transactions'})

product_stats['avg_price'] = product_stats['revenue'] / product_stats['quantity']
product_stats = product_stats.sort_values('revenue', ascending=False)

print("Product Performance:")
print(product_stats)

## 9. Advanced SQL Queries

In [ ]:
# Top 10 sales days
token = worker.execute("""
    SELECT date, SUM(revenue) as total_revenue, COUNT(*) as transactions
    FROM sales
    GROUP BY date
    ORDER BY total_revenue DESC
    LIMIT 10
""")
top_days = worker.fetch_results(token)

print("Top 10 Revenue Days:")
for i, (date, revenue, trans) in enumerate(top_days, 1):
    print(f"{i}. {date}: ${revenue:,.2f} ({trans} transactions)")

## 10. Category Performance Over Time

In [ ]:
# Monthly revenue by category
df['month'] = df['date'].dt.to_period('M')
monthly_category = df.groupby(['month', 'category'])['revenue'].sum().unstack()

# Plot
monthly_category.plot(kind='area', stacked=True, figsize=(14, 6), alpha=0.7)
plt.title('Monthly Revenue by Category', fontsize=14, fontweight='bold')
plt.xlabel('Month')
plt.ylabel('Revenue ($)')
plt.legend(title='Category', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

## 11. Export Results

In [ ]:
# Export summary to CSV
category_sales.to_csv('category_summary.csv')
product_stats.to_csv('product_summary.csv')

print("✅ Results exported to CSV files")

## 12. Cleanup

In [ ]:
# Close database connection
worker.close()
print("✅ Database connection closed")

## Next Steps

- Add your own data sources
- Create custom visualizations
- Implement machine learning models
- Build interactive dashboards
- Schedule automated reports